<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

[ваш текст]

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) и реализуйте полиморфизм с перекрытием и прегегрузкой методов, а также generic классы

<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [ ]:
using System;
using System.Collections.Generic;

class Program
{
    public void Main(string[] args)
    {
        OnlineOrder onlineOrder = new OnlineOrder(1, DateTime.Now, "qreft@mail.ru");
        PhysicalOrder physicalOrder = new PhysicalOrder(2, DateTime.Now, "Ленина, 23");
        SpecializedOrder specializedOrder = new SpecializedOrder(3, DateTime.Now, "Скидка 10%");
        InternationalOrder internationalOrder = new InternationalOrder(4, DateTime.Now, "qreftt@gmail.com", "Франция");
        DiscountedOrder discountedOrder = new DiscountedOrder(5, DateTime.Now, 15);

        onlineOrder.AddItem(new Item("Пицца", 1200));
        physicalOrder.AddItem(new Item("Пирог", 800));
        specializedOrder.AddItem(new Item("Пицца", 500));
        internationalOrder.AddItem(new Item("Пирог", 200));
        discountedOrder.AddItem(new Item("Пирог", 1000));

        physicalOrder.RemoveItem(new Item("Пирог", 800));

        Console.WriteLine($"Стоимость онлайн-заказа: {onlineOrder.CalculateTotal()}");
        Console.WriteLine($"Стоимость заказа: {physicalOrder.CalculateTotal()}");
        Console.WriteLine($"Стоимость специального заказа: {specializedOrder.CalculateTotal()}");
        Console.WriteLine($"Стоимость международного заказа: {internationalOrder.CalculateTotal()}");
        Console.WriteLine($"Стоимость заказа со скидкой: {discountedOrder.CalculateTotal()}");

        OrderManager<Order> orderManager = new OrderManager<Order>();
        orderManager.AddOrder(onlineOrder);
        orderManager.AddOrder(physicalOrder);
        orderManager.AddOrder(specializedOrder);
        orderManager.AddOrder(internationalOrder);
        orderManager.AddOrder(discountedOrder);

        orderManager.PrintAllOrders();
    }
}

public class Order
{
    public int OrderId { get; set; }
    public DateTime CreationDate { get; set; }
    public decimal TotalAmount { get; set; }
    public string OrderStatus { get; set; }
    protected List<Item> Items { get; set; }

    public Order(int orderId, DateTime creationDate)
    {
        OrderId = orderId;
        CreationDate = creationDate;
        Items = new List<Item>();
        OrderStatus = "Обрабатывается"; 
    }

    public virtual decimal CalculateTotal()
    {
        TotalAmount = 0;
        foreach (var item in Items)
        {
            TotalAmount += item.Price;
        }
        return TotalAmount;
    }

    public virtual void AddItem(Item item)
    {
        Items.Add(item);
    }

    public virtual void RemoveItem(Item item)
    {
        Items.Remove(item);
    }

    public void UpdateStatus(string newStatus)
    {
        OrderStatus = newStatus;
        Console.WriteLine($"Статус заказа {OrderId} обновлен: {newStatus}");
    }
}

public class Item
{
    public string Name { get; set; }
    public decimal Price { get; set; }

    public Item(string name, decimal price)
    {
        Name = name;
        Price = price;
    }
}

public class OnlineOrder : Order
{
    public string CustomerEmail { get; set; }

    public OnlineOrder(int orderId, DateTime creationDate, string customerEmail)
        : base(orderId, creationDate)
    {
        CustomerEmail = customerEmail;
    }

    public override void AddItem(Item item)
    {
        base.AddItem(item);
        Console.WriteLine($"'{item.Name}' добавлен в онлайн-заказ. Информация о доставке направлена на {CustomerEmail}.");
    }

    public void AddItem(Item item, string note)
    {
        base.AddItem(item);
        Console.WriteLine($"'{item.Name}' добавлен в онлайн-заказ с пометкой: {note}. Информация о доставке направлена на {CustomerEmail}.");
    }
}

public class PhysicalOrder : Order
{
    public string DeliveryAddress { get; set; }

    public PhysicalOrder(int orderId, DateTime creationDate, string deliveryAddress)
        : base(orderId, creationDate)
    {
        DeliveryAddress = deliveryAddress;
    }

    public override void RemoveItem(Item item)
    {
        base.RemoveItem(item);
        Console.WriteLine($"'{item.Name}' удален из физического заказа. Информация о возврате направлена на {DeliveryAddress}.");
    }

    public void RemoveItem(Item item, string reason)
    {
        base.RemoveItem(item);
        Console.WriteLine($"'{item.Name}' удален из физического заказа по причине: {reason}. Информация о возврате направлена на {DeliveryAddress}.");
    }
}

public class SpecializedOrder : Order
{
    public string SpecialConditions { get; set; }

    public SpecializedOrder(int orderId, DateTime creationDate, string specialConditions)
        : base(orderId, creationDate)
    {
        SpecialConditions = specialConditions;
    }

    public override decimal CalculateTotal()
    {
        decimal total = base.CalculateTotal();
        if (SpecialConditions.Contains("Скидка"))
        {
            total *= 0.9m;
        }
        return total;
    }
}

public class InternationalOrder : OnlineOrder
{
    public string Country { get; set; }

    public InternationalOrder(int orderId, DateTime creationDate, string customerEmail, string country)
        : base(orderId, creationDate, customerEmail)
    {
        Country = country;
    }

    public override void AddItem(Item item)
    {
        base.AddItem(item);
        Console.WriteLine($"'{item.Name}' добавлен в международный заказ. Информация о доставке направлена на {CustomerEmail}");
    }
}

public interface IDiscountable
{
    decimal ApplyDiscount(decimal total);
}

public class DiscountedOrder : Order, IDiscountable
{
    public decimal DiscountPercentage { get; set; }

    public DiscountedOrder(int orderId, DateTime creationDate, decimal discountPercentage)
        : base(orderId, creationDate)
    {
        DiscountPercentage = discountPercentage;
    }

    public decimal ApplyDiscount(decimal total)
    {
        return total * (1 - DiscountPercentage / 100);
    }

    public override decimal CalculateTotal()
    {
        decimal total = base.CalculateTotal();
        return ApplyDiscount(total);
    }
}

public class OrderManager<T> where T : Order
{
    private List<T> orders = new List<T>();

    public void AddOrder(T order)
    {
        orders.Add(order);
    }

    public void PrintAllOrders()
    {
        foreach (var order in orders)
        {
            Console.WriteLine($"Номер заказа: {order.OrderId}, Статус: {order.OrderStatus}, Стоимость: {order.CalculateTotal()}");
        }
    }
}

Program prog = new Program();
prog.Main([]);

'Пицца' добавлен в онлайн-заказ. Информация о доставке направлена на qreft@mail.ru.
'Пирог' добавлен в онлайн-заказ. Информация о доставке направлена на qreftt@gmail.com.
'Пирог' добавлен в международный заказ. Информация о доставке направлена на qreftt@gmail.com
'Пирог' удален из физического заказа. Информация о возврате направлена на Ленина, 23.
Стоимость онлайн-заказа: 1200
Стоимость заказа: 800
Стоимость специального заказа: 450.0
Стоимость международного заказа: 200
Стоимость заказа со скидкой: 850.00
Номер заказа: 1, Статус: Обрабатывается, Стоимость: 1200
Номер заказа: 2, Статус: Обрабатывается, Стоимость: 800
Номер заказа: 3, Статус: Обрабатывается, Стоимость: 450.0
Номер заказа: 4, Статус: Обрабатывается, Стоимость: 200
Номер заказа: 5, Статус: Обрабатывается, Стоимость: 850.00
